<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
    

<br>

<div style="font-weight: 700; font-size: 25px;"> [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course </div>

<br>

Auteurs: [Maria Sumarokova](https://www.linkedin.com/in/mariya-sumarokova-230b4054/) et [Yury Kashnitsky](https://www.linkedin.com/in/festline/). Traduit et édité par Gleb Filatov, Aleksey Kiselev, [Anastasia Manokhina](https://www.linkedin.com/in/anastasiamanokhina/), [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/), [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/) et [Ousmane Cissé](https://github.com/oussou-dev). Tout le contenu est distribué sous la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).</center>

<div style="font-weight: 700; font-size: 25px;"> <center> Mission n ° 3 (démo) </center></div>
    
<br>    
    
<div style="font-weight: 700; font-size: 20px;"> <center> Arbres de décision avec une "tâche jouet" (toy task) et le jeu de données UCI Adult </center></div>

    

La même mission sur [Kaggle Kernel](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + la [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution). Complétez les réponses dans le [formulaire Web](https://docs.google.com/forms/d/1wfWYYoqXTkZNOPy1wpewACXaj2MZjBdLOL58htGWYBA/edit).

Commençons par charger toutes les bibliothèques nécessaires:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10, 8)
import collections
from io import StringIO

import numpy as np
import pandas as pd
import pydotplus  # pip install pydotplus
import seaborn as sns
from ipywidgets import Image
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

### Partie 1. Jeu de données de jouet "Will They? Won't They?"

Votre objectif est de comprendre le fonctionnement des arbres de décision en résolvant un problème de jouet. Bien qu'un seul arbre de décision ne donne pas de résultats remarquables, d'autres algorithmes performants, tels que le boosting de gradient et les forêts aléatoires, sont basés sur la même idée. C'est pourquoi connaître le fonctionnement des arbres de décision peut être utile.

Nous allons passer à un exemple jouet de classification binaire - La personne A décide si elle ira à un deuxième rendez-vous avec la personne B. Cela dépendra de son apparence, de son éloquence, de sa consommation d’alcool (par exemple) et de combien d'argent elle a dépensé lors du premier rendez-vous

#### Création du jeu de données

In [ ]:
# Create dataframe with dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis=1)
    out.drop(feature_list, axis=1, inplace=True)
    return out


# Some feature values are present in train and absent in test and vice-versa.
def intersect_features(train, test):
    common_feat = list(set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [ ]:
features = ["Looks", "Alcoholic_beverage", "Eloquence", "Money_spent"]

#### Données d'entraînement

In [ ]:
df_train = {}
df_train["Looks"] = [
    "handsome",
    "handsome",
    "handsome",
    "repulsive",
    "repulsive",
    "repulsive",
    "handsome",
]
df_train["Alcoholic_beverage"] = ["yes", "yes", "no", "no", "yes", "yes", "yes"]
df_train["Eloquence"] = ["high", "low", "average", "average", "low", "high", "average"]
df_train["Money_spent"] = ["lots", "little", "lots", "little", "lots", "lots", "lots"]
df_train["Will_go"] = LabelEncoder().fit_transform(["+", "-", "+", "-", "-", "+", "+"])

df_train = create_df(df_train, features)
df_train

#### Données de test

In [ ]:
df_test = {}
df_test["Looks"] = ["handsome", "handsome", "repulsive"]
df_test["Alcoholic_beverage"] = ["no", "yes", "yes"]
df_test["Eloquence"] = ["average", "high", "average"]
df_test["Money_spent"] = ["lots", "little", "lots"]
df_test = create_df(df_test, features)
df_test

In [ ]:
# Some feature values are present in train and absent in test and vice-versa.
y = df_train["Will_go"]
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

In [ ]:
df_test

#### Dessinez un arbre de décision (à la main ou dans n’importe quel éditeur graphique) pour cet ensemble de données. Facultativement, vous pouvez également mettre en œuvre la construction de l'arbre et le dessiner ici.

1\. Quelle est l'entropie $S_0$ du système initial? Par états du système, nous entendons les valeurs de la caractéristique binaire "Will_go" - 0 ou 1 - deux états au total.

In [ ]:
# you code here

2 \. Partitionnons les données par la caractéristique "Looks_handsome". Qu'est-ce que l'entropie $S_1$ du groupe de gauche - celle avec "Looks_handsome". Quelle est l'entropie $S_2$ dans le groupe opposé? Quel est le gain d’information (IG) si on considère une telle partition?

In [ ]:
# you code here

#### Entraînez un arbre de décision en utilisant sklearn sur les données d'apprentissage. Vous pouvez choisir n'importe quelle profondeur pour l'arbre.

In [ ]:
# you code here

#### Supplément : affichez l'arbre résultant en utilisant graphviz. Vous pouvez utiliser pydot ou le [service Web](https://www.coolutils.com/ru/online/DOT-to-PNG) dot2png.

In [ ]:
# you code here

### Partie 2. Fonctions de calcul d'entropie et de gain d'information.

Prenons l'exemple d'échauffement suivant: nous avons 9 balles bleues et 11 balles jaunes. Laissez la balle avoir l'étiquette **1** si elle est bleue, **0** sinon.

In [ ]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

<img src="https://habrastorage.org/webt/mu/vl/mt/muvlmtd2njeqf18trbldenpqvnm.png">

Ensuite, divisez les balles en deux groupes:

<img src="https://habrastorage.org/webt/bd/aq/5w/bdaq5wi3c4feezaexponvin8wmo.png">

In [ ]:
# two groups
balls_left = [1 for i in range(8)] + [0 for i in range(5)]  # 8 blue and 5 yellow
balls_right = [1 for i in range(1)] + [0 for i in range(6)]  # 1 blue and 6 yellow

#### Implémenter une fonction pour calculer l'entropie de Shannon

In [ ]:
def entropy(a_list):
    # you code here
    pass

Quelques tests

In [ ]:
print(entropy(balls))  # 9 blue и 11 yellow
print(entropy(balls_left))  # 8 blue и 5 yellow
print(entropy(balls_right))  # 1 blue и 6 yellow
print(entropy([1, 2, 3, 4, 5, 6]))  # entropy of a fair 6-sided die

3 \. Quelle est l'entropie de l'état donné par la liste **balls_left**?

4 \. Quelle est l'entropie d'un dé non pipé? (où nous considérons un dé comme un système à 6 états également probables)?

In [ ]:
# calcul du gain d'information
def information_gain(root, left, right):
    """ root - initial data, left and right - two partitions of initial data"""

    # you code here
    pass

5 \. Quel est l’information que procure le fractionnement de l’ensemble de données initial en **balls_left** et **balls_right**?

In [ ]:
def best_feature_to_split(X, y):
    """Outputs information gain when splitting on best feature"""

    # you code here
    pass

#### Optionnel:
- Implémenter un algorithme de construction d'arbre de décision en appelant **best_feature_to_split** de manière récursive
- Tracer l'arbre résultant

### Part 3. Le jeu de données "Adult"

**Description du jeu de données:**

[Jeu de données](http://archive.ics.uci.edu/ml/machine-learning-databases/adult) UCI Adulte (inutile de le télécharger, nous en avons une copie dans le référentiel des cours): classifiez les personnes qui utilisent des données démographiques - qu’ils gagnent plus de 50 000 dollars par an ou non.

Description des caractéristiques :

- **Age** - caractéristique continue
- **Workclass** - caractéristique continue
- **fnlwgt** - poids final de l'objet, caractéristique continue
- **Education** - caractéristique catégorielle
- **Education_Num** - nombre d'années d'études, caractéristique continue
- **Martial_Status** - caractéristique catégorielle
- **Profession** - caractéristique catégorielle
- **Relation** - caractéristique catégorielle
- **Race** - caractéristique catégorielle
- **Sexe** - caractéristique catégorielle
- **Capital_Gain** - caractéristique continue
- **Capital_Loss** - caractéristique continue
- **Hours_per_week** - caractéristique continue
- **Pays** - caractéristique catégorielle

**Target** - Niveau de revenus, caractéristique catégorielle (binaire).

**Lecture des données d'entraînement et de test**

In [ ]:
data_train = pd.read_csv("../input/adult_train.csv")

In [ ]:
data_train.tail()

In [ ]:
data_test = pd.read_csv("../input/adult_test.csv")

In [ ]:
data_test.tail()

In [ ]:
# nécessaire pour supprimer les lignes avec des étiquettes incorrectes dans l'ensemble de données de test
data_test = data_test[
    (data_test["Target"] == " >50K.") | (data_test["Target"] == " <=50K.")
]

# encoder la variable cible en entier
data_train.loc[data_train["Target"] == " <=50K", "Target"] = 0
data_train.loc[data_train["Target"] == " >50K", "Target"] = 1

data_test.loc[data_test["Target"] == " <=50K.", "Target"] = 0
data_test.loc[data_test["Target"] == " >50K.", "Target"] = 1

**Analyse "primaire" des données**

In [ ]:
data_test.describe(include="all").T

In [ ]:
data_train["Target"].value_counts()

In [ ]:
fig = plt.figure(figsize=(25, 15))
cols = 5
rows = np.ceil(float(data_train.shape[1]) / cols)
for i, column in enumerate(data_train.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if data_train.dtypes[column] == np.object:
        data_train[column].value_counts().plot(kind="bar", axes=ax)
    else:
        data_train[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)

**Vérification des types de données**

In [ ]:
data_train.dtypes

In [ ]:
data_test.dtypes

Comme nous le voyons, dans les données de test, l’âge est traité comme un type **objet**. Nous devons résoudre ce problème.

In [ ]:
data_test["Age"] = data_test["Age"].astype(int)

Nous allons également transtyper (convertir) tous les éléments **float** en types **int** pour que les types restent cohérents entre nos données d'entraînement et de test.

In [ ]:
data_test["fnlwgt"] = data_test["fnlwgt"].astype(int)
data_test["Education_Num"] = data_test["Education_Num"].astype(int)
data_test["Capital_Gain"] = data_test["Capital_Gain"].astype(int)
data_test["Capital_Loss"] = data_test["Capital_Loss"].astype(int)
data_test["Hours_per_week"] = data_test["Hours_per_week"].astype(int)

**Complétez les données manquantes pour les caractéristiques continues avec leurs valeurs médianes, pour les caractéristiques catégorielles avec leur mode.**

In [ ]:
# choisir des caractéristiques catégorielles et continues à partir des données

categorical_columns = [
    c for c in data_train.columns if data_train[c].dtype.name == "object"
]
numerical_columns = [
    c for c in data_train.columns if data_train[c].dtype.name != "object"
]

print("categorical_columns:", categorical_columns)
print("numerical_columns:", numerical_columns)

In [ ]:
# nous voyons quelques valeurs manquantes
data_train.info()

In [ ]:
# compléter (remplacer) les données manquantes

for c in categorical_columns:
    data_train[c].fillna(data_train[c].mode()[0], inplace=True)
    data_test[c].fillna(data_train[c].mode()[0], inplace=True)

for c in numerical_columns:
    data_train[c].fillna(data_train[c].median(), inplace=True)
    data_test[c].fillna(data_train[c].median(), inplace=True)

In [ ]:
# plus de valeurs manquantes
data_train.info()

Nous allons coder des données factices pour certaines caractéristiques catégorielles: **Workclass**, **Education**, **Martial_Status**, **Profession**, **Relationship**, **Race**, **Sex**, **Country**. Cela peut être fait via la méthode pandas **get_dummies**

In [ ]:
data_train = pd.concat(
    [data_train[numerical_columns], pd.get_dummies(data_train[categorical_columns])],
    axis=1,
)

data_test = pd.concat(
    [data_test[numerical_columns], pd.get_dummies(data_test[categorical_columns])],
    axis=1,
)

In [ ]:
set(data_train.columns) - set(data_test.columns)

In [ ]:
data_train.shape, data_test.shape

**Il n'y a pas Holland (Hollande, Pays-bas) dans les données de test. Créer une nouvelle caractéristique de valeur zéro.**

In [ ]:
data_test["Country_ Holand-Netherlands"] = 0

In [ ]:
set(data_train.columns) - set(data_test.columns)

In [ ]:
data_train.head(2)

In [ ]:
data_test.head(2)

In [ ]:
X_train = data_train.drop(["Target"], axis=1)
y_train = data_train["Target"]

X_test = data_test.drop(["Target"], axis=1)
y_test = data_test["Target"]

#### 3.1 Arbre de décision sans réglage des paramètres

Entraînez un arbre de décision **(DecisionTreeClassifier) ​​** avec une profondeur maximale de 3, et évaluez la métrique de précision des données de test. Utilisez le paramètre **random_state = 17** pour la reproductibilité des résultats.

In [ ]:
# you code here
# tree =
# tree.fit

Faites une prédiction avec le modèle formé sur les données de test.

In [ ]:
# you code here
# tree_predictions = tree.predict

In [ ]:
# you code here
# accuracy_score

6 \. Quelle est la précision des données de test d'un arbre de décision avec une profondeur d'arbre maximale de 3 et **random_state = 17**?

#### 3.2 Arbre de décision avec réglage des paramètres

Entraînez un arbre de décision **(DecisionTreeClassifier, random_state = 17).** Recherchez la profondeur maximale optimale à l'aide de la validation croisée par 5-fold **(GridSearchCV)**.

In [ ]:
tree_params = {"max_depth": range(2, 11)}

locally_best_tree = GridSearchCV  # you code here

locally_best_tree.fit
# you code here

Entraînez un arbre de décision avec une profondeur maximale de 9 (il s'agit du meilleur **max_depth** dans mon cas) et calculez la précision des données de tests. Utilisez le paramètre **random_state = 17** pour la reproductibilité.

In [ ]:
# you code here
# tuned_tree =
# tuned_tree.fit
# tuned_tree_predictions = tuned_tree.predict
# accuracy_score

7\. Quelle est la précision de l'ensemble des données de tests d'un arbre de décision avec une profondeur maximale de 9 et **random_state = 17**?

#### 3.3 (Facultatif) Forêt aléatoire sans réglage de paramètre

Jetons un coup d'œil aux conférences à venir et essayons d'utiliser une forêt aléatoire pour notre tâche. Pour le moment, vous pouvez imaginer une forêt aléatoire comme un groupe d’arbres de décision formés sur des sous-ensembles légèrement différents des données d’entraînement.

Entraîner une forêt aléatoire **(RandomForestClassifier)**. Définissez le nombre d'arbres sur 100 et utilisez **random_state = 17**.

In [ ]:
# you code here
# rf =
# rf.fit # you code here

Faites des prévisions pour les données de test et évaluez leur précision.

In [ ]:
# you code here

#### 3.4 (Facultatif) Forêt aléatoire avec réglage des paramètres

Entraîner une forêt aléatoire **(RandomForestClassifier)**. Réglez la profondeur maximale et le nombre maximal de caractéristiques pour chaque arbre à l’aide de **GridSearchCV**.

In [ ]:
# forest_params = {'max_depth': range(10, 21),
#                 'max_features': range(5, 105, 20)}

# locally_best_forest = GridSearchCV # you code here

# locally_best_forest.fit # you code here

Faites des prévisions pour les données de test et évaluez leur précision.

In [ ]:
# you code here